In [2]:
import cv2
import os

def capture_images_without_corners(camera_index, output_folder, max_images=30):
    """
    USB ???? ???? ???? ?? (?? ?? ??)
    :param camera_index: USB ??? ??? (?: /dev/video0 -> 0)
    :param output_folder: ??? ???? ??? ??
    :param max_images: ??? ??? ?? ??
    """
    cap = cv2.VideoCapture(camera_index)
    if not cap.isOpened():
        print(f"Cannot open camera with index {camera_index}")
        return

    os.makedirs(output_folder, exist_ok=True)
    count = 0

    print("Press 'c' to capture an image, 'q' to quit.")
    while count < max_images:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        cv2.imshow("USB Camera", frame)
        key = cv2.waitKey(1)

        if key == ord('c'):  # 'c' ?? ?? ??? ??
            filepath = os.path.join(output_folder, f"calib_{count}.jpg")
            cv2.imwrite(filepath, frame)
            print(f"Saved: {filepath}")
            count += 1
        elif key == ord('q'):  # 'q' ?? ?? ??
            break

    cap.release()
    cv2.destroyAllWindows()

# ?? ??
#capture_images_without_corners(camera_index='/dev/video0', output_folder="./calibration_images/camera0", max_images=30)


In [21]:
import cv2
import numpy as np
import glob

def fisheye_calibrate(images_folder, pattern_size=(8, 6), square_size=0.03):
    # ?? ??? ??
    objp = np.zeros((pattern_size[1]*pattern_size[0], 3), np.float32)
    objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
    objp *= square_size  # ?? ??? ?? ??

    objpoints = []  # 3D ?? ???
    imgpoints = []  # 2D ??? ???

    images = glob.glob(f"{images_folder}/*.jpg")
    img_shape = None

    for fname in images:
        img = cv2.imread(fname)
        if img is None:
            print(f"???? ??? ? ????: {fname}")
            continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img_shape is None:
            img_shape = gray.shape[::-1]

        ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
        if ret:
            corners_refined = cv2.cornerSubPix(
                gray, corners, (3, 3), (-1, -1),
                (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
            )

            # ??? ??? ??? OpenCV ????? ?? ??
            objpoints.append(objp.reshape(-1, 1, 3).astype(np.float32))
            imgpoints.append(corners_refined.reshape(-1, 1, 2).astype(np.float32))
            print(f"Corners detected in: {fname}")
        else:
            print(f"Failed to detect corners in: {fname}")

    # ??? ??? ??
    if len(objpoints) != len(imgpoints) or len(objpoints) == 0:
        raise ValueError("Invalid data for calibration. Check your input images.")

    print(f"Number of object points: {len(objpoints)}")
    print(f"Number of image points: {len(imgpoints)}")

    # ?? ???? ??? ???? ?? ? ??? ?? ?? (????)
    print(f"objpoints[0] shape: {objpoints[0].shape}, dtype: {objpoints[0].dtype}")
    print(f"imgpoints[0] shape: {imgpoints[0].shape}, dtype: {imgpoints[0].dtype}")

    # ?????? ??
    K = np.zeros((3, 3))
    D = np.zeros((4, 1))
    rvecs = []  # ?? ??
    tvecs = []  # ?? ??

    flags = (cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC +
             cv2.fisheye.CALIB_FIX_SKEW)

    try:
        rms, K, D, rvecs, tvecs = cv2.fisheye.calibrate(
            objpoints, imgpoints, img_shape, K, D, rvecs, tvecs, flags,
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6)
        )
        print(f"Calibration RMS error: {rms}")
        print(f"Camera matrix (K):\n{K}")
        print(f"Distortion coefficients (D):\n{D}")
    except cv2.error as e:
        print(f"Fisheye calibration failed: {e}")
        return None, None

    return K, D

# ?? ??
K, D = fisheye_calibrate('./calibration_images/camera3', pattern_size=(8, 6), square_size=0.03)

import pickle

# ?????? ?? ?
calibration_data = {
    'K': K,
    'D': D
}

with open('calibration_data_camera3.pkl', 'wb') as f:
    pickle.dump(calibration_data, f)



Corners detected in: ./calibration_images/camera3/calib_19.jpg
Corners detected in: ./calibration_images/camera3/calib_20.jpg
Corners detected in: ./calibration_images/camera3/calib_11.jpg
Corners detected in: ./calibration_images/camera3/calib_10.jpg
Corners detected in: ./calibration_images/camera3/calib_17.jpg
Corners detected in: ./calibration_images/camera3/calib_6.jpg
Corners detected in: ./calibration_images/camera3/calib_21.jpg
Corners detected in: ./calibration_images/camera3/calib_23.jpg
Corners detected in: ./calibration_images/camera3/calib_7.jpg
Corners detected in: ./calibration_images/camera3/calib_8.jpg
Corners detected in: ./calibration_images/camera3/calib_16.jpg
Corners detected in: ./calibration_images/camera3/calib_28.jpg
Corners detected in: ./calibration_images/camera3/calib_18.jpg
Corners detected in: ./calibration_images/camera3/calib_15.jpg
Corners detected in: ./calibration_images/camera3/calib_29.jpg
Corners detected in: ./calibration_images/camera3/calib_25

In [19]:

import cv2
import numpy as np
import pickle

def undistort_fisheye(img, K, D, balance=0.0, new_dim=None):
    h, w = img.shape[:2]
    DIM = (w, h)  # (width, height)

    if new_dim is None:
        new_dim = DIM

    new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(
        K, D, DIM, np.eye(3), balance=balance, new_size=new_dim
    )

    map1, map2 = cv2.fisheye.initUndistortRectifyMap(
        K, D, np.eye(3), new_K, new_dim, cv2.CV_16SC2
    )

    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR)

    return undistorted_img

# ?????? ?? ??
with open('calibration_data_camera2.pkl', 'rb') as f:
    calibration_data = pickle.load(f)
    K = calibration_data['K']
    D = calibration_data['D']

# ?? ??? ??
test_img = cv2.imread('./calibration_images/camera2/homography.jpg')

if test_img is None:
    print("???? ??? ? ????.")
else:
    # ?? ?? ??
    undistorted = undistort_fisheye(test_img, K, D, balance=0.0)

    # ?? ??
    cv2.imshow('Original Image', test_img)
    cv2.imshow('Undistorted Image', undistorted)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # ?? ?? (??)
    cv2.imwrite('./calibration_images/camera2/undistorted_homography.jpg', undistorted)



In [20]:
capture_images_without_corners(camera_index='/dev/video4', output_folder="./calibration_images/camera1", max_images=30)

Press 'c' to capture an image, 'q' to quit.


In [21]:
capture_images_without_corners(camera_index='/dev/video8', output_folder="./calibration_images/camera2", max_images=30)

Press 'c' to capture an image, 'q' to quit.


In [22]:
capture_images_without_corners(camera_index='/dev/video25', output_folder="./calibration_images/camera3", max_images=30)

Press 'c' to capture an image, 'q' to quit.


In [23]:
capture_images_without_corners(camera_index='/dev/video0', output_folder="./calibration_images/camera0", max_images=30)

Press 'c' to capture an image, 'q' to quit.
